# **4. 계산그래프를 이용하여 퍼셉트론과 Backpropagation 구현하기**
목표 : 계산그래프를 이용하여, xor 함수를 근사하는 multilayer perceptron을 backpropagation으로 학습해봅시다.

---

In [ ]:
# coding: utf-8
import numpy as np
from collections import OrderedDict
import matplotlib.pyplot as plt

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = np.random.uniform(size=(input_size, hidden_size))
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = np.random.uniform(size=(hidden_size, output_size))
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Sigmoid1'] = Sigmoid()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Sigmoid2'] = Sigmoid()
        self.lastLayer = MSE()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads


# 인공적으로 입력과 출력 만들기 X : 입력 t : 정답(label)
# xor 데이터입니다.
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
t = np.array([[0], [1], [1], [0]])

network = TwoLayerNet(input_size=2, hidden_size=3, output_size=1)

epoch = 20000

learning_rate = 0.1

train_loss_list = [] # 결과 출력을 위한 코드
for i in range(epoch):
    # network의 기울기 계산 메서드인 gradient 호출
    grad = network.gradient(X, t)

    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(X, t)
    train_loss_list.append(loss) #결과 출력을 위한 코드
plt.plot(train_loss_list) # 결과 출력